In [2]:
obj= (1,2,3)
 
obj1= [ 6,7,8]

dt= { "c":1 , "C++":2 }

obj
obj1
dt

{'c': 1, 'C++': 2}

In [3]:
lit=[ 1.2,2,3.4,4]

obj= {"year":[1,2,3,4] ,"Month":[4.4,5.6,6.7],"Make":["c","c++","java"] }

obj.values()

obj.keys()

obj.get("year")
 
obj.pop("Make")

obj["year"]


[1, 2, 3, 4]

In [4]:
obj=989898

import sys 

sys.getrefcount(obj)

del obj


In [6]:
obj = 19898

obj1 = 19898

print(sys.getrefcount(obj))

print( id(obj) )



3
1903451318832


In [29]:
obj1=1

id(obj1)

1584098368

In [7]:
import pyspark

from pyspark.sql import SparkSession

from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [8]:
salesByModel = spark.read.csv('E:\\norway_new_car_sales_by_model.csv', header = True, inferSchema = True)

salesByModel

DataFrame[Year: int, Month: int, Make: string, Model: string, Quantity: int, Pct: double]

In [9]:
salesByMake = spark.read.format('csv') \
                        .option("inferSchema", True) \
                        .option("header", True) \
                        .option("sep", ',') \
                .load('E:\\norway_new_car_sales_by_model.csv')

In [10]:
# show(): displays the data frame values as it is.

# viz. ‘4’ tells to show only the top 4 rows, ‘False’ tells to show the complete value inside the cells.

In [11]:
salesByModel

DataFrame[Year: int, Month: int, Make: string, Model: string, Quantity: int, Pct: double]

In [12]:
salesByMake.show()

salesByMake.show(4)

salesByMake.show(4, False)

+----+-----+-----------+-----------------+--------+----+
|Year|Month|       Make|            Model|Quantity| Pct|
+----+-----+-----------+-----------------+--------+----+
|2007|    1|Volkswagen |Volkswagen Passat|    1267|10.0|
|2007|    1|    Toyota |      Toyota Rav4|     819| 6.5|
|2007|    1|    Toyota |   Toyota Avensis|     787| 6.2|
|2007|    1|Volkswagen |  Volkswagen Golf|     720| 5.7|
|2007|    1|    Toyota |   Toyota Corolla|     691| 5.4|
|2007|    1|   Peugeot |      Peugeot 307|     481| 3.8|
|2007|    1|     Skoda |    Skoda Octavia|     481| 3.8|
|2007|    1|    Toyota |     Toyota Yaris|     402| 3.2|
|2007|    1|      Ford |       Ford Focus|     400| 3.2|
|2007|    1|     Volvo |        Volvo V50|     346| 2.7|
|2007|    1|   Peugeot |      Peugeot 207|     320| 2.5|
|2007|    1|      Audi |          Audi A4|     269| 2.1|
|2007|    1|Volkswagen |Volkswagen Touran|     235| 1.9|
|2007|    1|      Ford |       Ford S-Max|     231| 1.8|
|2007|    1|      Opel |       

In [13]:
salesByMake.head()



Row(Year=2007, Month=1, Make='Volkswagen ', Model='Volkswagen Passat', Quantity=1267, Pct=10.0)

In [12]:
# printSchema(): prints the column names along with their data type and null indicator.

In [14]:
salesByMake.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Pct: double (nullable = true)



In [15]:
#Store the list of column names in a variable:
salesByMake.columns

['Year', 'Month', 'Make', 'Model', 'Quantity', 'Pct']

In [16]:
salesByMake.columns[0]

'Year'

In [17]:
salesByMake.count()  #get the count of rows

2694

In [18]:
df = salesByModel.select('Year', 'Make', 'Model', 'Quantity')

df.show(3)

+----+-----------+-----------------+--------+
|Year|       Make|            Model|Quantity|
+----+-----------+-----------------+--------+
|2007|Volkswagen |Volkswagen Passat|    1267|
|2007|    Toyota |      Toyota Rav4|     819|
|2007|    Toyota |   Toyota Avensis|     787|
+----+-----------+-----------------+--------+
only showing top 3 rows



In [18]:
# Change the name of existing columns(.withColumnRenamed())

In [20]:
# this imports sql functions that are extremely useful while dealing # with dataframes.

from pyspark.sql.functions import *

"""salesByModel.withColumnRenamed('OldColumnName1', 'NewColumnName1') \
            .withColumnRenamed('OldColumnName2', 'NewColumnName2') \
            .withColumnRenamed('OldColumnName3', 'NewColumnName3')"""

# Using alias is faster than using withColumnRenamed() function. So avoid to many withColumnRenamed

df2 =salesByModel.select(col('Year').alias('Newyear'), col('Make').alias('umake'))


In [21]:
df2.show()

+-------+-----------+
|Newyear|      umake|
+-------+-----------+
|   2007|Volkswagen |
|   2007|    Toyota |
|   2007|    Toyota |
|   2007|Volkswagen |
|   2007|    Toyota |
|   2007|   Peugeot |
|   2007|     Skoda |
|   2007|    Toyota |
|   2007|      Ford |
|   2007|     Volvo |
|   2007|   Peugeot |
|   2007|      Audi |
|   2007|Volkswagen |
|   2007|      Ford |
|   2007|      Opel |
|   2007|Volkswagen |
|   2007|      Audi |
|   2007|       BMW |
|   2007|      Opel |
|   2007|     Honda |
+-------+-----------+
only showing top 20 rows



In [22]:
# Add a new column to the data frame:

condition = when(col('Quantity')>=1000, "High").otherwise(when((col('Quantity')>=500) & (col('Quantity')<1000), "Medium").otherwise("Low"))

condition


Column<b'CASE WHEN (Quantity >= 1000) THEN High ELSE CASE WHEN ((Quantity >= 500) AND (Quantity < 1000)) THEN Medium ELSE Low END END'>

In [23]:
salesByModel.withColumn('NewCol1', col('Quantity')>1000 ).withColumn('NewCol2', condition).show()

+----+-----+-----------+-----------------+--------+----+-------+-------+
|Year|Month|       Make|            Model|Quantity| Pct|NewCol1|NewCol2|
+----+-----+-----------+-----------------+--------+----+-------+-------+
|2007|    1|Volkswagen |Volkswagen Passat|    1267|10.0|   true|   High|
|2007|    1|    Toyota |      Toyota Rav4|     819| 6.5|  false| Medium|
|2007|    1|    Toyota |   Toyota Avensis|     787| 6.2|  false| Medium|
|2007|    1|Volkswagen |  Volkswagen Golf|     720| 5.7|  false| Medium|
|2007|    1|    Toyota |   Toyota Corolla|     691| 5.4|  false| Medium|
|2007|    1|   Peugeot |      Peugeot 307|     481| 3.8|  false|    Low|
|2007|    1|     Skoda |    Skoda Octavia|     481| 3.8|  false|    Low|
|2007|    1|    Toyota |     Toyota Yaris|     402| 3.2|  false|    Low|
|2007|    1|      Ford |       Ford Focus|     400| 3.2|  false|    Low|
|2007|    1|     Volvo |        Volvo V50|     346| 2.7|  false|    Low|
|2007|    1|   Peugeot |      Peugeot 207|     320|

In [34]:
df.filter(trim(col('Make')) == 'Volkswagen').show()

+----+-----------+-----------------+--------+
|Year|       Make|            Model|Quantity|
+----+-----------+-----------------+--------+
|2007|Volkswagen |Volkswagen Passat|    1267|
|2007|Volkswagen |  Volkswagen Golf|     720|
|2007|Volkswagen |Volkswagen Touran|     235|
|2007|Volkswagen |  Volkswagen Polo|     218|
|2007|Volkswagen |Volkswagen Passat|     602|
|2007|Volkswagen |  Volkswagen Golf|     442|
|2007|Volkswagen |Volkswagen Touran|     189|
|2007|Volkswagen |  Volkswagen Polo|     162|
|2007|Volkswagen |Volkswagen Passat|     521|
|2007|Volkswagen |  Volkswagen Golf|     402|
|2007|Volkswagen |  Volkswagen Polo|     197|
|2007|Volkswagen |Volkswagen Touran|     194|
|2007|Volkswagen |Volkswagen Passat|     375|
|2007|Volkswagen |  Volkswagen Golf|     342|
|2007|Volkswagen |Volkswagen Touran|     229|
|2007|Volkswagen |  Volkswagen Polo|     167|
|2007|Volkswagen |  Volkswagen Golf|     669|
|2007|Volkswagen |Volkswagen Passat|     607|
|2007|Volkswagen |Volkswagen Toura

In [35]:
df.filter(trim(df.Make) == 'Toyota').show(2)

+----+-------+--------------+--------+
|Year|   Make|         Model|Quantity|
+----+-------+--------------+--------+
|2007|Toyota |   Toyota Rav4|     819|
|2007|Toyota |Toyota Avensis|     787|
+----+-------+--------------+--------+
only showing top 2 rows



In [33]:
#groupBy & orderBy: you should groupBy all the items and then apply aggregation functions such as min, max, sum, avg, count etc.
#Note: orderBy and Sort do the same thing.

In [34]:
salesByModel.groupBy('Make').agg(sum('Quantity').alias('SumQuantity'), avg('Pct').alias('AvgPct')).orderBy('SumQuantity').show()

+---------------+-----------+------------------+
|           Make|SumQuantity|            AvgPct|
+---------------+-----------+------------------+
|       Citroen |        129|  1.07625563157017|
|�Mercedes-Benz |        184|  1.53180153180153|
|          Saab |       1025|               2.0|
|        Subaru |       1713| 1.750612784170052|
|       Hyundai |       2254|1.2580827839895132|
|       Renault |       2820|1.3218679361421966|
|           Kia |       3096|1.1827709295905884|
|        Suzuki |       5431|1.3629232244177336|
|         Honda |       8197|1.8664246525408978|
|          Opel |       8990|1.4972583342311718|
| Mercedes-Benz |       9621|1.2681808332714253|
|         Tesla |      11334| 2.408628892942137|
|         Mazda |      19235|2.1048159388815213|
|       Peugeot |      22572|1.6274781293851686|
|           BMW |      22965|1.5253243958728175|
|          Audi |      28691|1.8213180296107934|
|    Mitsubishi |      29118| 2.404337010931026|
|         Skoda |   

In [35]:
salesByModel.join(salesByMake, trim(salesByMake["Make"]) == trim(salesByModel["Make"])).show()

+----+-----+-----------+-----------------+--------+----+----+-----+-----------+-----------------+--------+----------------+
|Year|Month|       Make|            Model|Quantity| Pct|Year|Month|       Make|            Model|Quantity|             Pct|
+----+-----+-----------+-----------------+--------+----+----+-----+-----------+-----------------+--------+----------------+
|2007|    1|Volkswagen |Volkswagen Passat|    1267|10.0|2017|    1|Volkswagen |Volkswagen Passat|     515|             3.9|
|2007|    1|Volkswagen |Volkswagen Passat|    1267|10.0|2017|    1|Volkswagen |  Volkswagen Golf|     738|             5.7|
|2007|    1|Volkswagen |Volkswagen Passat|    1267|10.0|2016|   12|Volkswagen |Volkswagen Passat|     299|             2.2|
|2007|    1|Volkswagen |Volkswagen Passat|    1267|10.0|2016|   12|Volkswagen |Volkswagen Tiguan|     650|             4.8|
|2007|    1|Volkswagen |Volkswagen Passat|    1267|10.0|2016|   12|Volkswagen |  Volkswagen Golf|     721|             5.3|
|2007|  

In [36]:
salesByMake

DataFrame[Year: int, Month: int, Make: string, Model: string, Quantity: int, Pct: double]

In [37]:
salesByModel

DataFrame[Year: int, Month: int, Make: string, Model: string, Quantity: int, Pct: double]

In [36]:
salesByModel.join(salesByMake, 
                  trim(salesByMake["Make"]) == trim(salesByModel["Make"]), 
                  how = "inner") \
            .select(salesByMake["Year"], 
                    salesByMake.Month, 
                    salesByModel["Pct"], 
                    salesByModel.Make) \
            .show(5)

+----+-----+----+-----------+
|Year|Month| Pct|       Make|
+----+-----+----+-----------+
|2017|    1|10.0|Volkswagen |
|2017|    1|10.0|Volkswagen |
|2016|   12|10.0|Volkswagen |
|2016|   12|10.0|Volkswagen |
|2016|   12|10.0|Volkswagen |
+----+-----+----+-----------+
only showing top 5 rows



In [37]:
from pyspark.sql.window import Window

Parition_Param = Window().partitionBy(col('Make')).orderBy('Pct')

salesByModel.withColumn('Sum',sum('Quantity').over(Parition_Param)).show(4)

+----+-----+------+------------+--------+-----------------+---+
|Year|Month|  Make|       Model|Quantity|              Pct|Sum|
+----+-----+------+------------+--------+-----------------+---+
|2015|    3|Skoda |Skoda Superb|      55|0.388445511688679| 55|
|2015|    4|Skoda |Skoda Superb|      61| 0.47723360976373|116|
|2016|    1|Skoda | Skoda Fabia|      63|0.573196251478482|179|
|2016|    2|Skoda | Skoda Fabia|      71|0.580919653084602|250|
+----+-----+------+------------+--------+-----------------+---+
only showing top 4 rows



In [38]:
df = salesByModel.withColumn('Sum',sum('Quantity').over(Parition_Param))